# Concatenate tweets

In this notebook, we will consolidate all the tweet files into a single compressed pickle file for further analysis. We have three main sets of data that we need to store: 

1. Data from January 2021.
2. Data from October 2021.
3. Data from April 28 to June 30.

Each of these samples corresponds to a specific moment relevant for our analysis. The October data is used for analyzing our community during election periods, specifically the regional elections in Colombia that took place in October 2019. The data from January 2021 represents the period three months before the "Paro Nacional," allowing us to track our community before the social outbreak. Finally, we have the data from the time of the "Paro Nacional," which will be the focal point of our analysis.

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [2]:
path = r"/mnt/disk2/Data"
pd.set_option("display.max_columns", None)

## Regional elections: October 2019

In [ ]:
# We create an empty aux list that will store the tweets.
tweets_aux = []
files_oct = glob(os.path.join(path, 'RawData','users_oct_19/*.csv'))
empties = []
for file in tqdm(files_oct):
    df = pd.read_csv(file, dtype = {'Author ID': int, 'Referenced Tweet Author ID': int})
    if df.empty:
        empties.append(file)
    else:
        tweets_aux.append(df)

# Finally, the tweet dataframe is established and tweets_aux is deleted.
tweets = pd.concat(tweets_aux)
del tweets_aux
tweets = tweets.sort_values('ID').reset_index(drop = True)

# Store results
tweets.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_oct19.gzip"), compression = "gzip")

## Before Paro Nacional: January 2021

We identify two users with their file corrupted: Usuario_82383620 and Usuario_2526574133

In [ ]:
# We create an empty aux list that will store the tweets.
tweets_aux = []
files_jan = glob(os.path.join(path, "RawData", "users_jan/*.csv"))

for file in tqdm(files_jan):
    df = pd.read_csv(file)
    if df.empty:
        empties.append(file)
    else:
        tweets_aux.append(df)

# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets_jan = pd.concat(tweets_aux)
del tweets_aux
tweets_jan = tweets_jan.sort_values('ID').reset_index(drop = True)

# Store results
# run sudo chmod 777 /mnt/disk2/Data/Tweets_DataFrames in bash if it is needed
tweets_jan.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_jan21.gzip"), compression = "gzip")

## Paro Nacional: April 28 - June 30 2021

In [ ]:
files_v1 = glob(os.path.join(path, 'RawData/Usuarios_V1/*.csv'))
len(files_v1)

In [ ]:
df = pd.read_csv(files_v1[0])
df['Reference Type']

In [ ]:
def unique_to_string(x):
    unique_values = x.unique()
    return ', '.join(map(str, unique_values))

user_information = df.groupby(['Author ID', 'Author Name']).agg({
                'Author Location': unique_to_string,
                'Author Description': unique_to_string,
                'Author Followers': lambda x: np.nanmean(x),
                'Author Following': lambda x: np.nanmean(x),
                'Author Tweets': lambda x: np.nanmax(x),
                'Author Verified': unique_to_string})

In [ ]:
user_information

In [ ]:
df_list = []
users_information = []

# cols = ['ID', 'Author ID', 'Author Name', 'Date', 'Text', 'Replies', 'Retweets', 'Favorites', 'Quotes', 'is Retweet?',
#            'Reply To User Name', 'Mentions', 'Referenced Tweet', 'Reference Type', 'Referenced Tweet Author ID']

problems = []

def unique_to_string(x):
    unique_values = x.unique()
    return ', '.join(map(str, unique_values))

# Counter and variable for keeping track of file names
count = 0 # Amount of Tweets
n = 0 # Number of Checkpoint

# Runtime 1 Hour!!!!!
for file in tqdm(files_v1):
    try:
        # df = pd.read_csv(file, usecols = cols)
        df = pd.read_csv(file)
        if df.empty:
            empties.append(file)
        else:
            pass
        # Fix some datatypes
        df[['Author Followers', 'Author Following', 'Author Tweets']] = df[['Author Followers', 'Author Following', 'Author Tweets']].map(lambda x: pd.to_numeric(x, errors = 'coerce'))
        df_list.append(df)
        count += len(df)

        # Save user information
        user_information = df.groupby(['Author ID', 'Author Name']).agg({
                'Author Location': unique_to_string,
                'Author Description': unique_to_string,
                'Author Followers': lambda x: np.nanmean(x),
                'Author Following': lambda x: np.nanmean(x),
                'Author Tweets': lambda x: np.nanmax(x),
                'Author Verified': unique_to_string})
        
        users_information.append(user_information)
        
        # If we reach or exceed 10 million rows, save the file and reset
        if count >= 7_000_000:
            n += 1
            concat_df = pd.concat(df_list)
            output_filename = f"tweets_paro_{n}.gzip"
            concat_df.to_pickle(os.path.join(path, f"Tweets_DataFrames/{output_filename}"), compression='gzip')
            
            # Reset counter and list
            count = 0
            df_list = []
            
    except (ValueError, KeyError) as e:
        problems.append(file)

problems

In [ ]:
empties

In [ ]:
# Save any remaining data after the loop
# Runtime 5 minutes
if df_list:
    n += 1
    concat_df = pd.concat(df_list)
    output_filename = f"tweets_paro_{n}.gzip"
    # If necessary, run "sudo chmod 777 Data/Tweets_DataFrames" in bash
    concat_df.to_pickle(os.path.join(path, f"Tweets_DataFrames/{output_filename}"), compression = 'gzip')

del df_list, concat_df

In [ ]:
# runtime 22 minutes
concat_users_information = pd.concat(users_information)
concat_users_information = concat_users_information.groupby(['Author ID', 'Author Name']) \
    .agg({'Author Location': unique_to_string,
            'Author Description': unique_to_string,
            'Author Followers': lambda x: np.nanmean(x),
            'Author Following': lambda x: np.nanmean(x),
            'Author Tweets': lambda x: np.nanmax(x),
            'Author Verified': unique_to_string})
concat_users_information.to_pickle(os.path.join(path, "Tweets_DataFrames/users_information.gzip"), 
                                   compression = 'gzip')

In [ ]:
# We should correct this
concat_users_information

In [4]:
tweets_paro = glob('/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_*')

tweets = pd.DataFrame()
for file in tqdm(tweets_paro):
    tweets_df = pd.read_pickle(file, compression = "gzip")
    tweets = pd.concat([tweets, tweets_df], axis = 0)
    del tweets_df
tweets.head()

100%|██████████| 7/7 [03:25<00:00, 29.31s/it]


,ID,Permalink,Author ID,Author Name,Author Location,Author Description,Author Followers,Author Following,Author Tweets,Author Profile Image,Author Verified,Date,Text,Replies,Retweets,Favorites,Quotes,is Retweet?,Reply To User Name,Mentions,Referenced Tweet,Reference Type,Referenced Tweet Author ID,Media URLs,Media Keys
0,1.409619e+18,/hmauriciojg/status/1409618955283668996,138377765.0,hmauriciojg,"Bucaramanga, Colombia",NaN,22.0,558.0,873.0,https://pbs.twimg.com/profile_images/154468480...,False,2021/06/28 16:05:23,@DanielSamperO A vida hp!!. @IvanDuque fue y s...,0.0,0.0,0.0,0.0,False,DanielSamperO,DanielSamperO IvanDuque petrogustavo,1.409586e+18,replied_to,1.348553e+08,NaN,NaN
1,1.409575e+18,/hmauriciojg/status/1409574993596452867,138377765.0,hmauriciojg,"Bucaramanga, Colombia",NaN,22.0,558.0,873.0,https://pbs.twimg.com/profile_images/154468480...,False,2021/06/28 13:10:41,@alejarojas_g A bueno de pronto si @petrogusta...,0.0,0.0,0.0,0.0,False,alejarojas_g,alejarojas_g petrogustavo,1.409192e+18,replied_to,1.131821e+09,NaN,NaN
2,1.409302e+18,/hmauriciojg/status/1409302180847292417,138377765.0,hmauriciojg,"Bucaramanga, Colombia",NaN,22.0,558.0,873.0,https://pbs.twimg.com/profile_images/154468480...,False,2021/06/27 19:06:38,@gabodelascasas Ahí la tiene https://t.co/2WJZ...,0.0,0.0,0.0,0.0,False,gabodelascasas,gabodelascasas,1.409298e+18,replied_to,6.233750e+07,https://pbs.twimg.com/media/E47Y3H4XMAMtHHu.jpg,3_1409302174933397507
3,1.407446e+18,/hmauriciojg/status/1407446306113691662,138377765.0,hmauriciojg,"Bucaramanga, Colombia",NaN,22.0,558.0,873.0,https://pbs.twimg.com/profile_images/154468480...,False,2021/06/22 16:12:03,@JOHANVE_LAND Deberías hacerle esa pregunta ta...,0.0,0.0,0.0,0.0,False,JOHANVE_LAND,JOHANVE_LAND petrogustavo,1.407171e+18,replied_to,5.766474e+08,NaN,NaN
4,1.407176e+18,/hmauriciojg/status/1407176029635067904,138377765.0,hmauriciojg,"Bucaramanga, Colombia",NaN,22.0,558.0,873.0,https://pbs.twimg.com/profile_images/154468480...,False,2021/06/21 22:18:04,@ClaraLopezObre @petrogustavo Que susto tan hp...,0.0,0.0,0.0,0.0,False,ClaraLopezObre,ClaraLopezObre petrogustavo,1.406750e+18,replied_to,1.268326e+08,NaN,NaN


In [14]:
tweets["Reference Type"] = tweets["Reference Type"].fillna("original tweet")

In [17]:
os.listdir(path)

['Daily_graphs',
 'ACLED Colombia (2018-01-01-2023-10-31).csv',
 'nodes.csv',
 'RawData',
 'Pickle',
 'Tweets_DataFrames',
 '3_Day_Graphs']

In [18]:
count_tweets = tweets.groupby(["Author ID", "Author Name", "Reference Type"]).size().reset_index(name = "n")
count_tweets.to_pickle(os.path.join(path, "count_tweets.gzip"), compression = "gzip")

In [19]:
count_tweets

,Author ID,Author Name,Reference Type,n
0,0.000000e+00,0,original tweet,108
1,1.000000e+00,0,original tweet,8
2,2.000000e+00,0,original tweet,11
3,3.000000e+00,0,original tweet,10
4,3.000000e+00,1,original tweet,1
...,...,...,...,...
137903,1.389769e+18,VaneLen18,retweeted,591
137904,1.389784e+18,kars0518,original tweet,3
137905,1.389784e+18,kars0518,quoted,13
137906,1.389784e+18,kars0518,replied_to,3


### Tweets Lite
We create a reduced version of the Paro data frame. This will have the same amount of rows but we will only store four columns: 'Author ID', 'Date', 'Reference Type', 'Referenced Tweet Author ID'.

In [ ]:
# Get just the columns that we need for the Graph construction
cols = ['Author ID','Author Name', 'Referenced Tweet Author ID', 'Date', 'Reference Type', 'Referenced Tweet']
tweets_lite = tweets[cols].reset_index(drop = True)
# Store results
# run sudo chmod 777 Data/Tweets_DataFrames in bash if it is needed
tweets_lite.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_lite.gzip"), compression = "gzip")

## Outputs

The output of this Notebook are stored "/mnt/disk2/Data/Tweets_DataFrames" and are listed below:

- **tweets_jan21.gzip**: Dataframe for the Tweets for our users during January of 2021. 3 Months before the Paro
- **tweets_oct19.gzip**: Dataframe for the Tweets for our users during October of 2019. Regional elections Period
- **tweets_paro_i.gzip**: 5 dataframes for the tweets of our users between April 28 to June 30 of 2021
- **tweets_lite.pkl**: Lite version of **tweets_Usuarios_V1.gzip** that contains just the colmns needed for the graph construction. Which is Author ID, Reference Type, Date and Retweet Author